## RNN (IMDB 영화 리뷰 데이터 긍/부정

In [ ]:
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/산기협_교육/Data/IMDB.csv', engine='python')
data = data[['review', 'sentiment']]
print(data.shape)

(50000, 2)


In [ ]:
data

# 특수문자는 tokenize를 하면 학습을 방해하는 요소임 -> 전처리 필요

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [ ]:
import re

data['review'] = data['review'].apply((lambda x:re.sub(r'[^A-za-z0-9\s]', '', x)))

In [ ]:
data

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production br br The filmin...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically theres a family where a little boy J...,negative
4,Petter Matteis Love in the Time of Money is a ...,positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,Bad plot bad dialogue bad acting idiotic direc...,negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,Im going to have to disagree with the previous...,negative


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

# 가장 많이 나오는 상위 2000개만 사용할 거임(전부 다 사용하면 숫자로 변환되는 단어들이 너무 많아)
max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ', lower=True, oov_token=True)
tokenizer.fit_on_texts(data['review'].values)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_sentence_len = 40
X = tokenizer.texts_to_sequences(data['review'].values)
X = pad_sequences(X, maxlen=max_sentence_len)
print('Data_shape : ', X.shape)

Data_shape :  (50000, 40)


In [ ]:
X[0]
# 토크나이저는 공백을 1로 처리함
# 위에서 2000개까지만 사용한다 했으니까 2000개까지가 숫자로 변환된거고 2000번까지 등장하지 않았다면 이보다 더 적을 수도 있음

array([   1,  753,  816,    1,  107,  660,   82, 1208,    1,  668,    6,
         64,  549,    5,  929, 1998,   40, 1208,  557,  146,    1,   23,
        197,  412,    1,   16,   49,    7,    1,    1,   44,   23,   69,
         76,    8, 1211,   16,  123,    1,  501], dtype=int32)

In [ ]:
from tensorflow.keras.layers import SpatialDropout1D, Embedding, Dense

embed_dim = 256
lstm_unit = 128

model = Sequential()
model.add(Embedding(max_features, embed_dim, input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_unit, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='softmax'))
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 40, 256)           512000    
                                                                 
 spatial_dropout1d_7 (Spatia  (None, 40, 256)          0         
 lDropout1D)                                                     
                                                                 
 lstm_3 (LSTM)               (None, 128)               197120    
                                                                 
 dense (Dense)               (None, 2)                 258       
                                                                 
Total params: 709,378
Trainable params: 709,378
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from sklearn.model_selection import train_test_split

Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5, random_state=50)

In [ ]:
from tensorflow.keras.optimizers import Adam

opt = Adam(lr=0.00003)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
history = model.fit(X_train, Y_train, epochs=10, batch_size=64, validation_split=0.1, verbose=1, shuffle=True)

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


352/352 [==============================] - 70s 185ms/step - loss: 0.6920 - accuracy: 0.5338 - val_loss: 0.6895 - val_accuracy: 0.5900
Epoch 2/10
352/352 [==============================] - 63s 178ms/step - loss: 0.6554 - accuracy: 0.6292 - val_loss: 0.5990 - val_accuracy: 0.6904
Epoch 3/10
352/352 [==============================] - 65s 184ms/step - loss: 0.5727 - accuracy: 0.7009 - val_loss: 0.5529 - val_accuracy: 0.7208
Epoch 4/10
352/352 [==============================] - 63s 179ms/step - loss: 0.5281 - accuracy: 0.7366 - val_loss: 0.5214 - val_accuracy: 0.7500
Epoch 5/10
352/352 [==============================] - 63s 179ms/step - loss: 0.5004 - accuracy: 0.7541 - val_loss: 0.5016 - val_accuracy: 0.7632
Epoch 6/10
352/352 [==============================] - 63s 179ms/step - loss: 0.4815 - accuracy: 0.7662 - val_loss: 0.4899 - val_accuracy: 0.7660
Epoch 7/10
352/352 [==============================] - 65s 184ms/step - loss: 0.4660 - accuracy: 0.7772 - val_loss: 0.4828 - val_accuracy: 0.7

### 예측

In [ ]:
sentence = 'I wanted to like this film and expected there to be a decent resolution to the breakdown of equilbrium but alas no, it left me feeling like Id wasted my time and the film makers had wasted their money'
sentence = tokenizer.texts_to_sequences([sentence])

sentence = pad_sequences(sentence, maxlen=max_sentence_len, dtype='int32', value=0)

In [ ]:
import numpy as np

sentiment = model.predict(sentence, batch_size=1, verbose=2)[0]

if(np.argmax(sentiment) == 0):
  print('negative')
elif (np.argmax(sentiment) == 1):
  print('positive')

1/1 - 0s - 300ms/epoch - 300ms/step
negative
